In [2]:
from bitstamp import BitStamp
bit = BitStamp()
bit.start()


connect_handler


In [13]:
bit.get_current_state()

{'amount': 0.00338297,
 'asks': [['19177.18000000', '0.99661703'], ['19204.86000000', '0.42962000']],
 'bids': [['19163.56000000', '0.02806342'], ['19162.99000000', '0.00093230']],
 'price': 19177.18}

In [14]:
bit.stop()

In [ ]:
json = {"bids": [["19017.09000000", "0.03570972"], ["19015.59000000", "0.73594500"], ["19012.68000000", "0.50000000"], ["19008.57000000", "0.01000000"], ["19001.12000000", "0.23655000"], ["19001.00000000", "0.94701570"], ["19000.95000000", "1.51944105"], ["18985.99000000", "0.09494014"], ["18985.95000000", "0.03473663"], ["18984.25000000", "1.19450829"], ["18978.30000000", "0.08717900"], ["18970.61000000", "1.00000000"], ["18966.83000000", "0.13296000"], ["18966.82000000", "0.43154652"], ["18950.02000000", "0.45919012"], ["18950.00000000", "0.01519815"], ["18949.23000000", "0.01739128"], ["18935.27000000", "0.01739128"], ["18931.07000000", "0.47310000"], ["18931.06000000", "0.02664000"], ["18931.05000000", "0.54886806"], ["18929.65000000", "2.32480000"], ["18921.31000000", "0.01739128"], ["18913.32000000", "1.00000000"], ["18905.00000000", "0.00952125"], ["18900.00000000", "2.82742774"], ["18883.00000000", "0.00042050"], ["18875.61000000", "0.05535906"], ["18874.00000000", "0.00042070"], ["18873.36000000", "0.00255752"], ["18869.08000000", "0.42962000"], ["18865.00000000", "0.00042091"], ["18860.00000000", "0.10055551"], ["18859.19000000", "0.05185522"], ["18858.00000000", "0.00320562"], ["18856.00000000", "0.00042111"], ["18855.23000000", "0.04851170"], ["18850.00000000", "0.50000000"], ["18847.00000000", "0.06953742"], ["18838.00000000", "0.00042151"], ["18836.29000000", "1.20927953"], ["18829.00000000", "0.00042171"], ["18825.00000000", "0.04000000"], ["18820.00000000", "0.00042191"], ["18815.00000000", "0.01339063"], ["18811.59000000", "0.30000000"], ["18811.00000000", "0.00042211"], ["18810.01000000", "3.13682696"], ["18810.00000000", "0.16340000"], ["18808.35000000", "0.20000000"], ["18807.00000000", "0.02000000"], ["18806.00000000", "0.82388599"], ["18805.00000000", "0.01000000"], ["18803.32000000", "0.00086700"], ["18802.00000000", "0.00042232"], ["18801.01000000", "0.10128996"], ["18801.00000000", "2.23000000"], ["18800.97000000", "0.27605703"], ["18800.00000000", "1.02652095"], ["18793.00000000", "0.00042252"], ["18792.00000000", "0.20000000"], ["18789.00000000", "0.10000000"], ["18787.00000000", "0.02000000"], ["18786.00000000", "0.05000000"], ["18784.00000000", "0.00042272"], ["18782.31000000", "25.10000000"], ["18780.23000000", "0.01151330"], ["18780.00000000", "0.41004409"], ["18775.00000000", "0.00042292"], ["18770.00000000", "0.01340000"], ["18768.00000000", "0.00325326"], ["18767.00000000", "0.02000000"], ["18766.00000000", "0.00042313"], ["18765.00000000", "85.00000000"], ["18764.28000000", "4.29620000"], ["18757.00000000", "0.00042333"], ["18756.01000000", "1.00557153"], ["18750.00000000", "3.43713744"], ["18748.00000000", "0.00042353"], ["18747.00000000", "0.02000000"], ["18746.95000000", "0.50000000"], ["18740.00000000", "0.01340000"], ["18739.00000000", "0.00042374"], ["18730.00000000", "0.53432676"], ["18729.27000000", "0.03303380"], ["18727.00000000", "0.02000000"], ["18725.00000000", "0.04000000"], ["18721.00000000", "0.00369370"], ["18719.00000000", "0.10000000"], ["18715.80000000", "0.01488800"], ["18712.00000000", "0.00042435"], ["18709.57000000", "0.00051292"], ["18707.00000000", "0.02000000"], ["18703.46000000", "1.99616648"], ["18703.00000000", "0.00042455"], ["18700.00000000", "24.80466756"], ["18699.46000000", "0.00052878"], ["18697.62000000", "0.00051290"], ["18694.00000000", "0.00042476"], ["18688.87000000", "25.10000000"]], "asks": [["19024.95000000", "0.16169316"], ["19032.69000000", "0.32819262"], ["19050.00000000", "0.02558830"], ["19057.54000000", "0.06649860"], ["19060.00000000", "0.10000000"], ["19063.98000000", "0.23655000"], ["19064.00000000", "1.20140668"], ["19065.45000000", "0.05460000"], ["19065.46000000", "25.00000000"], ["19071.11000000", "0.00041898"], ["19072.00000000", "6.87567746"], ["19072.01000000", "0.01739128"], ["19072.89000000", "0.00050316"], ["19080.19000000", "0.00041878"], ["19085.98000000", "0.01092000"], ["19085.99000000", "0.01739128"], ["19089.27000000", "0.00041858"], ["19089.89000000", "0.72420461"], ["19090.82000000", "12.43375600"], ["19097.29000000", "1.17219900"], ["19098.35000000", "0.00041838"], ["19099.96000000", "0.01739128"], ["19100.00000000", "17.07792020"], ["19100.17000000", "0.09000000"], ["19105.68000000", "0.02000000"], ["19106.38000000", "0.10000000"], ["19107.43000000", "0.00041818"], ["19112.86000000", "0.07915913"], ["19113.94000000", "0.01739128"], ["19115.24000000", "0.20000000"], ["19116.32000000", "1.20000000"], ["19121.98000000", "0.42962000"], ["19125.59000000", "0.00041778"], ["19127.92000000", "0.01739128"], ["19128.00000000", "0.02940848"], ["19133.52000000", "1.07017506"], ["19134.68000000", "0.00041759"], ["19140.00000000", "0.04000000"], ["19141.51000000", "0.00039174"], ["19141.52000000", "0.00028731"], ["19141.53000000", "0.00033954"], ["19141.54000000", "0.00039175"], ["19141.55000000", "0.02000000"], ["19142.52000000", "0.00028717"], ["19142.53000000", "0.00028720"], ["19142.54000000", "0.01739128"], ["19143.76000000", "0.00041739"], ["19145.46000000", "0.02000000"], ["19146.00000000", "0.00033946"], ["19146.01000000", "0.00039165"], ["19146.02000000", "0.00073115"], ["19146.03000000", "0.00028716"], ["19146.04000000", "0.00039166"], ["19146.05000000", "0.00039167"], ["19146.06000000", "0.00033938"], ["19146.07000000", "0.00039168"], ["19146.08000000", "0.00221946"], ["19146.09000000", "0.00028720"], ["19146.10000000", "0.00028723"], ["19146.11000000", "0.00033937"], ["19146.12000000", "0.00060049"], ["19146.13000000", "0.00028723"], ["19146.14000000", "0.00062662"], ["19146.15000000", "0.00028724"], ["19146.16000000", "0.00039160"], ["19146.17000000", "0.00033939"], ["19146.18000000", "0.00028725"], ["19146.19000000", "0.00039165"], ["19146.20000000", "0.00039162"], ["19146.21000000", "0.00039166"], ["19147.93000000", "0.00039162"], ["19147.94000000", "0.00028722"], ["19147.95000000", "0.00033937"], ["19147.96000000", "0.00033945"], ["19147.97000000", "0.00033945"], ["19147.98000000", "0.47338722"], ["19147.99000000", "0.00033943"], ["19148.00000000", "3.00000000"], ["19149.90000000", "0.00039160"], ["19149.91000000", "0.00028713"], ["19149.92000000", "0.00039156"], ["19149.93000000", "0.00140976"], ["19149.94000000", "0.00028705"], ["19149.95000000", "0.00039156"], ["19149.96000000", "0.00033933"], ["19149.97000000", "0.00065264"], ["19149.98000000", "0.00033933"], ["19149.99000000", "0.00028706"], ["19150.00000000", "49.21601107"], ["19150.50000000", "0.04000000"], ["19151.01000000", "1.27389300"], ["19151.60000000", "0.00028712"], ["19152.05000000", "0.02000000"], ["19152.34000000", "0.00039154"], ["19152.35000000", "0.02000000"], ["19152.84000000", "0.00041719"], ["19155.00000000", "0.30000000"], ["19156.38000000", "0.00033925"], ["19156.39000000", "0.00039151"], ["19156.40000000", "0.00028707"]]}

In [ ]:
json["aaa"] = 123

In [15]:
import gym

In [16]:
env_name = 'CartPole-v1'
env = gym.make(env_name)

[2017-12-17 12:38:02,729] Making new env: CartPole-v1


In [19]:
env.action_space.n

2

In [5]:
reward = 3
_time_fee

In [6]:
reward = -_time_fee

{'flat': {2, 3}, 'long': {2, 3}, 'ordened': {3, 6, 432}, 'short': {2, 3}}